Yaprak Sınıflandırması
Gerekli Kütüphanelerin Eklenmesi

In [133]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder


Test ve Eğitim Verilerinin Okunması


In [134]:
train=pd.read_csv("C:/Users/omerk/Documents/GitHub/yapayzeka/hafta8/yaprakveriseti/train.csv")
test=pd.read_csv("C:/Users/omerk/Documents/GitHub/yapayzeka/hafta8/yaprakveriseti/test.csv")

Sınıfların Belirlenmesi ve Etiketlenmesi


In [135]:
label_encoder=LabelEncoder().fit(train.species)
labels=label_encoder.transform(train.species)
classes =list(label_encoder.classes_)

Verilerin Hazırlanması , Özellik ve Sınıf Sayısının Belirlenmesi 

In [136]:
train=train.drop(["id","species"],axis=1)
test=test.drop(["id"],axis=1)
nb_features=192
nb_classes=len(classes)

Eğitim Verilerinin Standartlaştırılması

In [137]:
from sklearn.preprocessing import StandardScaler
scaler=StandardScaler().fit(train.values)
train=scaler.transform(train.values)

Eğitim Verisinin Eğitim ve Doğrulama İçin Ayarlanması

In [138]:
from sklearn.model_selection import train_test_split
x_train , x_valid , y_train , y_valid = train_test_split(train, labels ,test_size=0.1)

Etiketlerin kategorileştirilmesi


In [139]:
from tensorflow.keras.utils import to_categorical
y_train=to_categorical(y_train)
y_valid=to_categorical(y_valid)

x_train=np.array(x_train).reshape(891,192,1)
x_valid=np.array(x_valid).reshape(99,192,1)

Modelin oluşturulması 


In [140]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Activation,Conv1D,Dropout,MaxPooling1D,Flatten

model=Sequential()
model.add(Conv1D(512,1,input_shape=(nb_features,1)))
model.add(Activation("relu"))
model.add(MaxPooling1D(2))
model.add(Conv1D(256,1))
model.add(Activation("relu"))
model.add(MaxPooling1D(2))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(2048,activation="relu"))
model.add(Dense(1024,activation="relu"))
model.add(Dense(nb_classes,activation="softmax"))
model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d_6 (Conv1D)           (None, 192, 512)          1024      
                                                                 
 activation_6 (Activation)   (None, 192, 512)          0         
                                                                 
 max_pooling1d_6 (MaxPooling  (None, 96, 512)          0         
 1D)                                                             
                                                                 
 conv1d_7 (Conv1D)           (None, 96, 256)           131328    
                                                                 
 activation_7 (Activation)   (None, 96, 256)           0         
                                                                 
 max_pooling1d_7 (MaxPooling  (None, 48, 256)          0         
 1D)                                                  

Ağın Derlenmesi

In [141]:
model.compile(loss="categorical_crossentropy",optimizer="adam",metrics=["accuracy"])

Modelin Eğitilmesi


In [142]:
model.fit(x_train,y_train,epochs=15,validation_data=(x_valid,y_valid))

Epoch 1/15
28/28 [==============================] - 14s 471ms/step - loss: 3.6656 - accuracy: 0.1942 - val_loss: 1.6830 - val_accuracy: 0.5657
Epoch 2/15
28/28 [==============================] - 13s 459ms/step - loss: 1.0467 - accuracy: 0.7508 - val_loss: 1.0318 - val_accuracy: 0.7576
Epoch 3/15
28/28 [==============================] - 13s 457ms/step - loss: 0.3483 - accuracy: 0.8990 - val_loss: 0.8260 - val_accuracy: 0.7475
Epoch 4/15
28/28 [==============================] - 13s 453ms/step - loss: 0.2526 - accuracy: 0.9428 - val_loss: 0.6059 - val_accuracy: 0.8384
Epoch 5/15
28/28 [==============================] - 14s 500ms/step - loss: 0.1161 - accuracy: 0.9607 - val_loss: 0.7594 - val_accuracy: 0.8687
Epoch 6/15
28/28 [==============================] - 14s 492ms/step - loss: 0.1009 - accuracy: 0.9731 - val_loss: 0.6332 - val_accuracy: 0.8485
Epoch 7/15
28/28 [==============================] - 14s 499ms/step - loss: 0.1052 - accuracy: 0.9764 - val_loss: 0.5718 - val_accuracy: 0.8384

Ortalama Değerlerin Gösterilmesi


In [ ]:
accuracy yerine f1score, kesinlik(precision) , duyarlılık(sensivity) ve özgüllük(specificity)

In [ ]:
print(("Ortalama Eğitim Kaybı: ",np.mean(model.history.history["loss"])))
print(("Ortalama Eğitim Başarımı: ",np.mean(model.history.history["accuracy"])))
print(("Ortalama Doğrulama Kaybı: ",np.mean(model.history.history["val_loss"])))
print(("Ortalama Doğrulama Başarımı: ",np.mean(model.history.history["val_accuracy"])))

Değerlerin grafik üzerinde gösterilmesi



In [ ]:
import matplotlib.pyplot as plt
fig,(ax1,ax2)=plt.subplots(2,1,figsize=(15,15))
ax1.plot(model.history.history['loss'],color='g',label="Eğitim Kaybı")
ax1.plot(model.history.history['val_loss'],color='y',label="Doğrulama kaybı")
ax1.set_xticks(np.arange(15,75,15))
ax2.plot(model.history.history['accuracy'],color='b',label="Eğitim Başarımı")
ax2.plot(model.history.history['val_accuracy'],color='r',label="Doğrulama Başarımı")
ax2.set_xticks(np.arange(15,75,15))
plt.legend()
plt.show()